In [ ]:
import pandas as pd

In [ ]:
# 依次读入数据并查看缺失值情况
hyinfo_df = pd.read_excel('附件1-会员信息表.xlsx',sheet_name=0)

In [ ]:
hyinfo_df.isna().sum()

In [ ]:
xsls_df = pd.read_excel('附件2-销售流水表.xlsx')
xsls_df.isna().sum()

In [ ]:
hyxf_df = pd.read_excel('附件3-会员消费明细表.xlsx')
hyxf_df.isna().sum()

In [ ]:
spinfo_df = pd.read_excel('附件4-商品信息表.xlsx')
spinfo_df.isna().sum()

In [ ]:
# 删除spinfo_df中有空值的行
nullidx = spinfo_df[spinfo_df.isna().any(axis=1)].index

In [ ]:
spinfo_df = spinfo_df.drop(index=nullidx)

In [ ]:
# 发现商品信息表中很多进价为0的商品，推测是这些专柜已经撤离，所以删除进价为0的商品
# zeroidx = spinfo_df[spinfo_df['sj']==0.0].index

In [ ]:
# spinfo_df = spinfo_df.drop(index=zeroidx)

In [ ]:
# 连接该店会员信息表和会员消费表
df1 = hyinfo_df.merge(hyxf_df, on='kh')

In [ ]:
df1.columns

In [ ]:
# 删除重复的卡号列，收音机号，柜组名称（太多缺失）
df1 = df1.drop(columns=['kh.1', 'syjh', 'gzmc'])

In [ ]:
# 增加优惠列，用售价减去消费金额
df1['yh'] = df1['sj'] - df1['je']

In [ ]:
# 计算年龄
df1['csny'] = df1['csny'].astype('datetime64[Y]')
df1['nl'] = [(2018 - date.year) for date in list(df1['csny'])]

In [ ]:
df1 = df1.drop(columns=['csny'])

In [ ]:
# 计算消费时间
df1['dtime'] = df1['dtime'].astype('datetime64')
df1['dayofweek'] = df1['dtime'].dt.dayofweek
df1['hour'] = df1['dtime'].dt.hour

In [ ]:
df1

In [ ]:
# 保存为二进制的pickle格式，方便下次快速读取
import pickle
with open('（本店）会员信息和会员消费.pkl', 'wb') as f:
    pickle.dump(df1, f)

In [ ]:
# 非会员数据是从消费流水中删除会员消费的单据号
hydjh = list(hyxf_df['djh'])
hydjhidx = []
for i,row in xsls_df.iterrows():
    if row['djh'] in hydjh:
        hydjhidx.append(i)

In [ ]:
df2 = xsls_df.drop(index=hydjhidx)

In [ ]:
df2['dtime'] = df2['dtime'].astype('datetime64')
df2['dayofweek'] = df2['dtime'].dt.dayofweek
df2['hour'] = df2['dtime'].dt.hour

In [ ]:
df2 = df2.drop(columns='syjh')

In [ ]:
df2['yh'] =  df2['sj'] - df2['je']

In [ ]:
# 保存pickle
with open('非会员消费.pkl', 'wb') as f:
    pickle.dump(df2, f)

In [ ]:
# 整理所有会员的购买信息（包括非本商场会员）
df3 = hyxf_df.merge(spinfo_df[['spbm', 'lbbm', 'ppbm', 'csbm', 'wsjj', 'hsjj', 'jxs', 'xxs', 'jlrq']], on='spbm', how='inner')

In [ ]:
['kh', 'dtime', 'spbm', 'sl', 'sj', 'je', 'spmc', 'jf', 'syjh', 'djh',
       'gzbm', 'gzmc', 'lbbm', 'ppbm', 'csbm', 'wsjj', 'hsjj', 'jxs', 'xxs',
       'jlrq']

In [69]:
df3['dtime'] = df3['dtime'].astype('datetime64')
df3 = df3.drop(columns=['syjh'])

In [67]:
# 保存pickle
with open('所有会员消费.pkl', 'wb') as f:
    pickle.dump(df3, f)